## Summary Video Generation 

In [1]:
import os
import pandas as pd
import h5py
import cv2
import numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.editor import concatenate_videoclips
import os    
# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
# os.environ["FFMPEG_BINARY"] = "/path/to/custom/ffmpeg"

#### )) Util method

In [2]:
def video_map(key, info_df):
    if info_df is None:
        return key
    name = info_df.loc[int(key.split('_')[1])-1]['video_id']
    return name

#### )) Pick summary frames

In [3]:
def pick_frames(args):

    encoded_summ = args['encode_summary_path']
    # s='DSN/models/summe/result4.h5'
    with h5py.File(encoded_summ, 'r') as f:
        for key in f.keys():       
            if key == 'mean_fm':
                continue
               
            if 'video_info' in args:
                info_df = pd.read_csv(args['video_info'], sep='\t')
                video_name = video_map(key, info_df)
            else:
                video_name = f[key+'/video_name'][()].decode().split('.')[0]
                
            videofile=args['video_folder']+'/'+video_name+'.mp4'
            summ_folder=args['output_folder']+'/'+video_name+'/frames/'
            
            print(key, video_name) 
            
            if not os.path.exists(summ_folder):
                os.makedirs(summ_folder)
            machine_summary = f[key+'/machine_summary'][()]
            camera = cv2.VideoCapture(videofile)
            times = 0
            c=0
            while times<len(machine_summary):
                
                res, image = camera.read()
                if not res:
                    print(times)
                    break
                if machine_summary[times] == 1:
                    cv2.imwrite(summ_folder + str(c).zfill(6) + '.jpg', image)
                    c+=1
                times += 1
            
            camera.release()


#### )) Stich-back videos

In [4]:
def generate_video(args):
    sum_folders = os.listdir(args['output_folder'])
    for file in sum_folders:
        p=args['output_folder']+'/'+file+'/'
        frms = p+"frames"
        print(p)
        cmd = "ffmpeg -y -r 24 -i " + frms+"/%06d.jpg -c:v libx264 -r 24 -pix_fmt yuv420p "+args['output_folder']+'/'+file+".mp4"
        # print(cmd)
        command = ( cmd
         )
        # "ffmpeg -y -r 15 -i  vasnet_results/%06d.jpg -c:v libx264 -r 30 -pix_fmt yuv420p vasnet_results/ui.mp4"
        os.system(command)
# execute the command using os.system



#### )) Clip-out video with Audio

In [5]:
def clipout_with_audio(args):
    encoded_summ = args['encode_summary_path']
    summ_folder = args['output_folder']
    print(summ_folder)
    if not os.path.exists(summ_folder):
        os.makedirs(summ_folder)
        
    with h5py.File(encoded_summ, 'r') as f:
        for key in f.keys():  
            print(key)
            path2video=args['video_folder']+f[key+'/video_name'][()].decode()
            video = VideoFileClip(path2video)
            fps = video.fps
            machine_summary = f[key+'/machine_summary'][()]
            start, end = 0,0
            summ = list(machine_summary)
            l = len(summ)
            clips=[]
            while end!=l-1:
                if 1 not in summ[start:]:
                    break
                start = summ[start:].index(1)+start
                
                if 0 not in summ[start:]:
                    end = len(summ)
                else:
                    end = summ[start:].index(0)+start
                # clip=video.subclip(start/fps, (end-1)/fps)
                clip = video.cutout(0,start/fps).cutout( (end-1)/fps, video.duration)
                clip.fps = fps
                clips.append(clip)
                start=end
                # print(start)
            print(len(clips))
            final_clip = concatenate_videoclips(clips)
            final_clip.fps = fps
            final_clip.write_videofile(summ_folder+'/'+f[key+'/video_name'][()].decode())    

            

#### )) Genrate Summaries

*On Prebuilt features*

In [5]:
args={
    'encode_summary_path':'../Model/vasnet/results/tvsum_results.h5',
    'video_folder':'../data/Public datasets/ydata-tvsum50-v1_1/video',
    'video_info':'../data/Public datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-info.tsv',
    'output_folder': 'generated_summary/tvsum_summs'
    
}

In [48]:
pick_frames(args)

video_1 AwmHb44_ouw
video_10 akI8YFjEmUw
video_11 i3wAGJaaktw
video_12 Bhxk-O1Y7Ho
video_13 0tmA_C6XwfM
video_14 3eYKfiOEJNs
video_15 xxdtq8mxegs
video_16 WG0MBPpPC6I
video_17 Hl-__g2gn_A
video_18 Yi4Ij2NM7U4
video_19 37rzWOQsNIw
video_2 98MoyGZKHXc
video_20 LRw_obCPUt0
video_21 cjibtmSLxQ4
video_22 b626MiF1ew4
video_23 XkqCExn6_Us
video_24 GsAD1KT1xo8
video_25 PJrm840pAUI
video_26 91IHQYk1IQM
video_27 RBCABdttQmI
video_28 z_6gVvQb2d0
video_29 fWutDQy1nnY
video_3 J0nA4VgnoCo
video_30 4wU_LUjG5Ic
video_31 VuWGsYPqAX8
video_32 JKpqYvAdIsw
video_33 xmEERLqJ2kU
video_34 byxOvuiIJV0
video_35 _xMr-HKMfVA
video_36 WxtbjNsCQ8A
video_37 uGu_10sucQo
video_38 EE-bNr36nyA
video_39 Se3oxnaPsz0
video_4 gzDbaEs1Rlg
video_40 oDXZc0tZe04
video_41 qqR6AEXwxoQ
video_42 EYqVtI9YWJA
video_43 eQu1rNs0an0
video_44 JgHubY5Vw3Y
video_45 iVt07TCkFM0
video_46 E11zDS9XGzg
video_47 NyBmCxDoHJU
video_48 kLxoNp-UchI
video_49 jcoYJXDG9sw
video_5 XzYM3PfTM4w
video_50 -esJrBWj2d8
video_6 HT5vyqe0Xaw
video_7 sTEELN-vY30

In [49]:
generate_video(args)

ffmpeg -y -r 30 -i generated_summary/tvsum_summs/-esJrBWj2d8/frames/%06d.jpg -c:v libx264 -r 30 -pix_fmt yuv420p generated_summary/tvsum_summs/-esJrBWj2d8.mp4
ffmpeg -y -r 30 -i generated_summary/tvsum_summs/0tmA_C6XwfM/frames/%06d.jpg -c:v libx264 -r 30 -pix_fmt yuv420p generated_summary/tvsum_summs/0tmA_C6XwfM.mp4
ffmpeg -y -r 30 -i generated_summary/tvsum_summs/37rzWOQsNIw/frames/%06d.jpg -c:v libx264 -r 30 -pix_fmt yuv420p generated_summary/tvsum_summs/37rzWOQsNIw.mp4
ffmpeg -y -r 30 -i generated_summary/tvsum_summs/3eYKfiOEJNs/frames/%06d.jpg -c:v libx264 -r 30 -pix_fmt yuv420p generated_summary/tvsum_summs/3eYKfiOEJNs.mp4
ffmpeg -y -r 30 -i generated_summary/tvsum_summs/4wU_LUjG5Ic/frames/%06d.jpg -c:v libx264 -r 30 -pix_fmt yuv420p generated_summary/tvsum_summs/4wU_LUjG5Ic.mp4
ffmpeg -y -r 30 -i generated_summary/tvsum_summs/91IHQYk1IQM/frames/%06d.jpg -c:v libx264 -r 30 -pix_fmt yuv420p generated_summary/tvsum_summs/91IHQYk1IQM.mp4
ffmpeg -y -r 30 -i generated_summary/tvsum_sum

KeyboardInterrupt: 

*On Custom features*

In [7]:
args={
    'encode_summary_path':'../Model/vasnet/results/tvsum_results_normal.h5',
    'video_folder':'../data/Public datasets/ydata-tvsum50-v1_1/video',
    'video_info':'../data/Public datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-info.tsv',
    'output_folder': 'generated_summary/tvsum_summs_normal'
    
}

In [10]:
pick_frames(args)

video_1 AwmHb44_ouw
video_10 akI8YFjEmUw


KeyboardInterrupt: 

In [58]:
generate_video(args)

generated_summary/tvsum_summs_normal/-esJrBWj2d8/
generated_summary/tvsum_summs_normal/0tmA_C6XwfM/
generated_summary/tvsum_summs_normal/37rzWOQsNIw/
generated_summary/tvsum_summs_normal/3eYKfiOEJNs/


*Test Features*

In [46]:
# args={
#     'encode_summary_path':'../Model/HMT/results/test1_results_normal.h5',
#     'video_folder':'../data/Custom dataset/',
#     # 'video_info':'../data/Public datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-info.tsv',
#     'output_folder': 'generated_summary/test1_summs_normal'
    
# }

args={
    'encode_summary_path':'../Model/vasnet/results/customSetResults.h5',
    'video_folder':'../data/Custom dataset/',
    # 'video_info':'../data/Public datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-info.tsv',
    'output_folder': 'generated_summary/custom_test'
    
}

In [52]:
clipout_with_audio(args)

generated_summary/custom_test
video_1
Moviepy - Building video generated_summary/custom_test/0tmA_C6XwfM.mp4.
MoviePy - Writing audio in 0tmA_C6XwfMTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
fps3: 25.0
Moviepy - Writing video generated_summary/custom_test/0tmA_C6XwfM.mp4

fps2: 25.0
Fps1: 25.0


Moviepy - Done !
Moviepy - video ready generated_summary/custom_test/0tmA_C6XwfM.mp4
video_2
Moviepy - Building video generated_summary/custom_test/3eYKfiOEJNs.mp4.
MoviePy - Writing audio in 3eYKfiOEJNsTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
fps3: 25.0
Moviepy - Writing video generated_summary/custom_test/3eYKfiOEJNs.mp4

fps2: 25.0
Fps1: 25.0


Moviepy - Done !
Moviepy - video ready generated_summary/custom_test/3eYKfiOEJNs.mp4
video_3
Moviepy - Building video generated_summary/custom_test/ehsaas.mp4.
MoviePy - Writing audio in ehsaasTEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready generated_summary/custom_test/ehsaas.mp4
video_4
Moviepy - Building video generated_summary/custom_test/test1.mp4.
MoviePy - Writing audio in test1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
fps3: 24.0
Moviepy - Writing video generated_summary/custom_test/test1.mp4

fps2: 24.0
Fps1: 24.0


Moviepy - Done !
Moviepy - video ready generated_summary/custom_test/test1.mp4
video_5
Moviepy - Building video generated_summary/custom_test/video_2.mp4.
MoviePy - Writing audio in video_2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
fps3: 24.0
Moviepy - Writing video generated_summary/custom_test/video_2.mp4

fps2: 24.0
Fps1: 24.0


Moviepy - Done !
Moviepy - video ready generated_summary/custom_test/video_2.mp4


In [18]:
pick_frames(args)

video_1 0tmA_C6XwfM
video_2 3eYKfiOEJNs
video_3 ehsaas
video_4 test1
video_5 video_2


In [21]:
generate_video(args)

generated_summary/custom_test/0tmA_C6XwfM/
generated_summary/custom_test/3eYKfiOEJNs/
generated_summary/custom_test/ehsaas/


KeyboardInterrupt: 

---------------------------

In [53]:
video = VideoFileClip('../data/Custom dataset/0tmA_C6XwfM.mp4')
dur = video.duration
clip=video.cutout(0,8.5).cutout(8.8,dur)
clip.audio

In [33]:
dur

141.34

### Custom

In [19]:
args={
    'encode_summary_path':'../../models/Trirar/TrirarSegOri/Custom_set.h5',
    'video_folder':'../data/Custom dataset/',
    # 'video_info':'../data/Public datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-info.tsv',
    'output_folder': 'generated_summary/custom_test'
    
}

In [20]:
clipout_with_audio(args)

generated_summary/custom_test
video_1


t:  11%|███████▌                                                           | 57/507 [30:34<00:02, 218.20it/s, now=None]

18
Moviepy - Building video generated_summary/custom_test/0tmA_C6XwfM.mp4.
MoviePy - Writing audio in 0tmA_C6XwfMTEMP_MPY_wvf_snd.mp3



t:  11%|███████▌                                                           | 57/507 [30:34<00:02, 218.20it/s, now=None]

MoviePy - Done.
Moviepy - Writing video generated_summary/custom_test/0tmA_C6XwfM.mp4



...


t:  11%|███████▌                                                           | 57/507 [30:40<00:02, 218.20it/s, now=None]

Moviepy - Done !
Moviepy - video ready generated_summary/custom_test/3eYKfiOEJNs.mp4
video_3


t:  11%|███████▌                                                           | 57/507 [30:41<00:02, 218.20it/s, now=None]

1
Moviepy - Building video generated_summary/custom_test/ehsaas.mp4.
MoviePy - Writing audio in ehsaasTEMP_MPY_wvf_snd.mp3


ValueError: Number of samples, -107334, must be non-negative.

In [6]:
!pip uninstall -y moviepy decorator 


Found existing installation: moviepy 1.0.3
Uninstalling moviepy-1.0.3:
  Successfully uninstalled moviepy-1.0.3
Found existing installation: decorator 4.4.2
Uninstalling decorator-4.4.2:
  Successfully uninstalled decorator-4.4.2


In [ ]:
!pip install moviepy